# Heisenberg XXX model

$$ H_{\text{TFIM}} = \sum_{j=1}^{L}(X_jX_{j+1}+Y_jY_{j+1}+Z_jZ_{j+1}) $$

pip install qrisp 

https://www.qrisp.eu


In [1]:
import networkx as nx
import numpy as np

# Create a graph
L = 10
G = nx.Graph()
G.add_edges_from([(k,k+1) for k in range(L-1)]) 
#nx.draw(G, with_labels=True)

## Define Heisenberg Hamiltonian

In [2]:
from qrisp.operators import X, Y, Z

def create_heisenberg_hamiltonian(G):
    H = sum(X(i)*X(j)+Y(i)*Y(j)+Z(i)*Z(j) for (i,j) in G.edges())
    return H

H = create_heisenberg_hamiltonian(G)
print(H)

X(0)*X(1) + X(1)*X(2) + X(2)*X(3) + X(3)*X(4) + X(4)*X(5) + X(5)*X(6) + X(6)*X(7) + X(7)*X(8) + X(8)*X(9) + Y(0)*Y(1) + Y(1)*Y(2) + Y(2)*Y(3) + Y(3)*Y(4) + Y(4)*Y(5) + Y(5)*Y(6) + Y(6)*Y(7) + Y(7)*Y(8) + Y(8)*Y(9) + Z(0)*Z(1) + Z(1)*Z(2) + Z(2)*Z(3) + Z(3)*Z(4) + Z(4)*Z(5) + Z(5)*Z(6) + Z(6)*Z(7) + Z(7)*Z(8) + Z(8)*Z(9)


## Calculate eigenvalues clasically

In [3]:
from scipy.sparse.linalg import eigsh
matrix = H.to_sparse_matrix()
eigenvalues, eigenvectors = eigsh(matrix, k=2, which='SA')
E0 = eigenvalues[0]
v0 = eigenvectors[:,0]

## VQE warm start 

* 2 layers of problem specific Ansatz: https://arxiv.org/abs/2108.08086, https://qrisp.eu/reference/Algorithms/vqe/vqeProblems/heisenberg.html

In [4]:
from qrisp import QuantumVariable
from qrisp.vqe.problems.heisenberg import *

vqe = heisenberg_problem(G,1,0)
U_0 = vqe.train_function(QuantumVariable(G.number_of_nodes()),depth=1,max_iter=100)

qv = QuantumVariable(G.number_of_nodes())
U_0(qv)
E_0 = H.get_measurement(qv,precision=0.01)
print(E_0)

psi = qv.qs.statevector_array()

np.abs(np.dot(psi.conj().transpose(),v0))**2

/Users/renezander/Desktop/Qrisp/src/qrisp/operators/qubit/qubit_operator.py:1570: UserWarning: DeprecationWarning: This method will no longer be supported in a later release of Qrisp. Instead please migrate to .expectation_value.
  warnings.warn("DeprecationWarning: This method will no longer be supported in a later release of Qrisp. Instead please migrate to .expectation_value.")


-16.322811934632377                                                                  


np.float64(0.8786313665939549)

## QPE

Recall that $n+\lceil \log(2+\frac{1}{2\epsilon})\rceil$ precision qubits for QPE yield an estimate accurate to $n$ bits with probability $1-\epsilon$.

In [10]:
from qrisp import QPE

qv = QuantumVariable(G.number_of_nodes())
U_0(qv)

# Hamiltonian simulation via first order Suzuki-Trotter formula with 5 steps 
# Note: 5 steps are insufficient to obtain the ground state energy within error 0.1 (10 steps are sufficent)
exp_H = H.trotterization(order=1,forward_evolution=False)

qpe_res = QPE(qv,exp_H,precision=5,kwargs={"steps":5},iter_spec=True)

qc = qpe_res.qs.compile()
tqc = qc.transpile(basis_gates=["cx","u"])

print(qc.count_ops())
print(tqc.count_ops())
print(tqc.num_qubits())
print(tqc.depth())

{'x': 10, 'h': 5298, 'p': 42, 'cx': 16791, 'rz': 8380, 'sx': 2790, 'sx_dg': 2790, 'QFT_dg': 1, 'gphase': 1}
{'u3': 16673, 'cx': 16817}
15
12579


In [14]:
results = qpe_res.get_measurement(precompiled_qc=qc)
sorted_results= dict(sorted(results.items(), key=lambda item: item[1], reverse=True))

In [15]:
sorted_results

{0.34375: 0.5975380492390153,
 0.875: 0.11682766344673108,
 0.375: 0.09758804823903522,
 0.78125: 0.04929901401971961,
 0.3125: 0.031619367612647754,
 0.84375: 0.014409711805763885,
 0.125: 0.01126977460450791,
 0.8125: 0.010859782804343915,
 0.09375: 0.008999820003599928,
 0.40625: 0.00893982120357593,
 0.28125: 0.007419851602967942,
 0.625: 0.005579888402231956,
 0.90625: 0.005569888602227956,
 0.4375: 0.0035199296014079722,
 0.75: 0.003289934201315974,
 0.6875: 0.003269934601307974,
 0.25: 0.002299954000919982,
 0.46875: 0.002129957400851983,
 0.9375: 0.0019799604007919845,
 0.15625: 0.0019199616007679848,
 0.1875: 0.0019199616007679848,
 0.65625: 0.0017199656006879863,
 0.21875: 0.0015499690006199878,
 0.71875: 0.0015399692006159878,
 0.0625: 0.0015099698006039882,
 0.5: 0.0013099738005239896,
 0.53125: 0.00126997460050799,
 0.96875: 0.0011999760004799903,
 0.03125: 0.0009599808003839924,
 0.0: 0.0009499810003799925,
 0.59375: 0.000889982200355993,
 0.5625: 0.0008499830003399932}

## Find ground state engergy

* Exact solution: -17.03214083
* QPE results are modulo $2\pi$: $-17.03214083 + 4\pi = -4.465770215640827$
* Subtract $4\pi$ from solution

In [18]:
import numpy as np

for phi,prob in list(sorted_results.items())[:5]:
    E_qpe = 2*np.pi*(phi-1) # Results are modulo 2*pi, therefore subtract 2*pi
    E_qpe -= 4*np.pi 
    print("Energy: " + str(E_qpe) + " Probability: " + str(prob))

Energy: -16.689710972195776 Probability: 0.5975380492390153
Energy: -13.351768777756622 Probability: 0.11682766344673108
Energy: -16.493361431346415 Probability: 0.09758804823903522
Energy: -13.940817400304708 Probability: 0.04929901401971961
Energy: -16.886060513045138 Probability: 0.031619367612647754
